In [8]:
import gzip
import os
import numpy as np

def load_custom_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            with gzip.open(os.path.join(folder_path, filename), 'rb') as file:
                data = np.frombuffer(file.read(), dtype=np.uint8, offset=16)
            images.append(data.reshape(-1, 28, 28))
    return images

# Provide the path to the folder containing your custom dataset files
custom_dataset_folder = 'D:\\Capstone Project (Test)\\Mixed Output Data'

# Load all images in the custom dataset folder
custom_images = load_custom_images(custom_dataset_folder)


BadGzipFile: Not a gzipped file (b'\x89P')

In [1]:
import os
from PIL import Image

# Specify the path to your custom dataset folder
dataset_folder = 'D:\\Capstone Project (Test)\\Mixed Output Data'

# Initialize empty lists to store images and labels
images = []
labels = []

# List all files in the dataset folder
files = os.listdir(dataset_folder)

# Loop through the files and load the images
for file in files:
    if file.endswith(".png"):
        # Load the image using PIL (Python Imaging Library)
        image = Image.open(os.path.join(dataset_folder, file))
        
        # Append the image object to the images list
        images.append(image)
        
        # Extract the label from the file name or any other source
        label = 0  # You need to implement logic to extract the label from the file
        labels.append(label)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
  File "C:\Users\Dell\AppData\Local\Temp\ipykernel_18400\2015176919.py", line 18, in <module>
    image = Image.open(os.path.join(dataset_folder, file))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py", line 3218, in open
OSError: [Errno 24] Too many open files: 'D:\\Capstone Project (Test)\\Mixed Output Data\\A-4886.png'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\int

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Capstone Project (Test)\\Train DataA-0.png'

In [2]:
import numpy as np
from numpy import expand_dims, zeros, ones, vstack, random
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from keras.preprocessing.image import load_img, img_to_array
from matplotlib import pyplot
from os import listdir

# Define the standalone discriminator model
def define_discriminator(in_shape=(28, 28, 3)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # Foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # Upsample to 14x14
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # Upsample to 28x28
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (7, 7), activation='sigmoid', padding='same'))
    return model

# Define the combined generator and discriminator model for updating the generator
def define_gan(g_model, d_model):
    # Make weights in the discriminator not trainable
    d_model.trainable = False
    # Connect them
    model = Sequential()
    # Add generator
    model.add(g_model)
    # Add the discriminator
    model.add(d_model)
    # Compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# Load and preprocess your custom dataset
def load_custom_dataset(path_to_images):
    images = []
    for filename in listdir(path_to_images):
        img = load_img(path_to_images + '\\' + filename, target_size=(28, 28))  # Use '\\' for Windows
        img = img_to_array(img)
        images.append(img)
    X = np.array(images)
    X = X.astype('float32') / 255.0
    return X

# Select real samples
def generate_real_samples(dataset, n_samples):
    ix = random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y

# Generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    x_input = random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# Use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y

# Create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    for i in range(n * n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(examples[i])
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

# Evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real * 100, acc_fake * 100))
    save_plot(x_fake, epoch)
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)

# Train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=5, n_batch=32):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            d_loss, _ = d_model.train_on_batch(X, y)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        if (i+1) % 5 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

# Size of the latent space
latent_dim = 100
# Create the discriminator
d_model = define_discriminator()
# Create the generator
g_model = define_generator(latent_dim)
# Create the GAN
gan_model = define_gan(g_model, d_model)
# Load your custom dataset
path_to_custom_dataset = 'D:\\Capstone Project (Test)\\Mixed Output Data'
dataset = load_custom_dataset(path_to_custom_dataset)


MemoryError: Unable to allocate 3.26 GiB for an array with shape (372451, 28, 28, 3) and data type float32